In [99]:
import pandas as pd
import sys
sys.path.append('../')

from vocal_patterns.ml_logic.data import get_data

data = get_data()
data.head()

,path,exercise,technique,filename
0,/Users/jake/code/jchaselubitz/vocal_patterns/d...,arpeggios,slow_piano,f7_arpeggios_f_slow_piano_u.wav
1,/Users/jake/code/jchaselubitz/vocal_patterns/d...,scales,fast_forte,m8_scales_f_fast_forte_i.wav
2,/Users/jake/code/jchaselubitz/vocal_patterns/d...,arpeggios,slow_forte,m9_arpeggios_c_slow_forte_o.wav
3,/Users/jake/code/jchaselubitz/vocal_patterns/d...,arpeggios,slow_piano,f4_arpeggios_c_slow_piano_a.wav
4,/Users/jake/code/jchaselubitz/vocal_patterns/d...,arpeggios,fast_piano,m6_arpeggios_c_fast_piano_o.wav


In [115]:
# df_with_slices = pd.DataFrame(columns=["spectrogram", "exercise", "technique"])
sample_rate = 22050

print(df_with_slices)
def scaled_spectrogram(wave_trunc, sr):
    mel_spectrogram = librosa.feature.melspectrogram(y=wave_trunc, sr=sr)
    power_to_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
    min_value = np.min(power_to_db)
    max_value = np.max(power_to_db)

    # NORMALIZING gray array so that all values lie between [0, 1]
    normalized_spectrogram = (power_to_db - min_value) / (max_value - min_value)

    return normalized_spectrogram


def slice_waves(waveform, sr, snippet_duration=4, overlap=3):
    # Calculate the frame size and hop length
    frame_size = int(snippet_duration * sr)
    hop_length = int((snippet_duration - overlap) * sr)

    # Get the total number of snippets
    num_snippets = int(np.floor((len(waveform) - frame_size) / hop_length)) + 1

    new_4sec_arrays = []
    # Slice the audio into snippets
    for i in range(num_snippets):
        start_sample = i * hop_length
        end_sample = start_sample + frame_size
        snippet = waveform[start_sample:end_sample]
        # Append the snippet to the list
        new_4sec_arrays.append(snippet)

    return new_4sec_arrays

data_list = []
for index, row in data.iterrows():
    exercise = row['exercise']
    technique = row['technique']
    waveform, sr = librosa.load(row["path"], sr=sample_rate)
    slice_waveforms = slice_waves(waveform, sr=sample_rate)
    for w in slice_waveforms:
        normalized_spectrogram = scaled_spectrogram(w, sr=sample_rate)
        data_list.append({'spectrogram': normalized_spectrogram, 'exercise': exercise, 'technique': technique})

df_with_slices = pd.DataFrame(data_list)
df_with_slices

# for index, row in data.head(400).iterrows():
#     exercise = row['exercise']
#     technique = row['technique']
#     waveform, sr = librosa.load(row["path"], sr=sample_rate)
#     slice_waveforms = slice_waves(waveform, sr=sample_rate)
#     for w in slice_waveforms:
#         normalized_spectrogram = scaled_spectrogram(w, sr)
#         # new_row = {'spectrogram': w, 'exercise': exercise, 'technique': technique}
#         df = pd.DataFrame({'spectrogram': w, 'exercise': exercise, 'technique': technique})
#         df_with_slices = pd.concat([df_with_slices, df ], ignore_index=True)

# return df_with_slices

                                            spectrogram   exercise   technique
0     [[0.23180476, 0.2892316, 0.3014535, 0.26719418...  arpeggios  slow_piano
1     [[0.2858866, 0.26965076, 0.22097206, 0.2691479...  arpeggios  slow_piano
2     [[0.40495643, 0.36227638, 0.28256026, 0.288222...  arpeggios  slow_piano
3     [[0.43902367, 0.38572454, 0.3691454, 0.3202114...  arpeggios  slow_piano
4     [[0.4204014, 0.3536261, 0.27059212, 0.2587041,...  arpeggios  slow_piano
...                                                 ...        ...         ...
2102  [[0.4991149, 0.48423472, 0.46026325, 0.4134513...  arpeggios        belt
2103  [[0.4130972, 0.3740099, 0.35848665, 0.33925018...  arpeggios        belt
2104  [[0.47385788, 0.44830495, 0.39947405, 0.437893...  arpeggios        belt
2105  [[0.46448112, 0.42889538, 0.36465287, 0.280395...  arpeggios        belt
2106  [[0.45262694, 0.44250888, 0.4067912, 0.4384994...  arpeggios        belt

[2107 rows x 3 columns]


,spectrogram,exercise,technique
0,"[[0.23180476, 0.2892316, 0.3014535, 0.26719418...",arpeggios,slow_piano
1,"[[0.2858866, 0.26965076, 0.22097206, 0.2691479...",arpeggios,slow_piano
2,"[[0.40495643, 0.36227638, 0.28256026, 0.288222...",arpeggios,slow_piano
3,"[[0.43902367, 0.38572454, 0.3691454, 0.3202114...",arpeggios,slow_piano
4,"[[0.4204014, 0.3536261, 0.27059212, 0.2587041,...",arpeggios,slow_piano
...,...,...,...
15661,"[[0.34217557, 0.40158233, 0.43335074, 0.413529...",scales,slow_forte
15662,"[[0.3840975, 0.33667487, 0.3557849, 0.39009723...",scales,slow_forte
15663,"[[0.40539208, 0.44813833, 0.43008795, 0.409095...",scales,slow_forte
15664,"[[0.3574254, 0.40332788, 0.4217616, 0.4130477,...",scales,slow_forte


In [124]:
# df_with_slices["exercise"].value_counts()
df_with_slices.memory_usage(deep=True).sum() / (1024**2)

1327.2931413650513

In [87]:
X = data.drop(columns=['exercise', "technique", "filename"])
y = data[['exercise']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [88]:
y_test[:2677]

,exercise
1643,arpeggios
382,arpeggios
2553,other
2213,arpeggios
661,arpeggios
...,...
2511,arpeggios
872,scales
1386,scales
1924,arpeggios


In [82]:
import sys
sys.path.append('../')

from vocal_patterns.interface.main import predict



In [83]:
X_test["path"].head()

2001    /Users/jake/code/jchaselubitz/vocal_patterns/d...
2597    /Users/jake/code/jchaselubitz/vocal_patterns/d...
969     /Users/jake/code/jchaselubitz/vocal_patterns/d...
1929    /Users/jake/code/jchaselubitz/vocal_patterns/d...
1280    /Users/jake/code/jchaselubitz/vocal_patterns/d...
Name: path, dtype: object

In [89]:
# X_test["path"][574]
import librosa

waveform, sr = librosa.load(X_test["path"][382], sr=22050)
X = waveform


In [69]:
import numpy as np
predict(X)
# predicted_indices = np.argmax(predict(X), axis=1)
# predicted_indices


Load latest model from local registry...

Load latest model from disk...
✅ Model loaded from local disk
1/1 [==============================] - 0s 102ms/step


1